# Comprehensive Investment Advisory Workflow Using CrewAI


In this notebook, we illustrate a sophisticated investment advisory system built using **CrewAI**. The workflow combines multiple AI agents, each equipped with specialized tools, to:
1. Gather financial data and news about a company.
2. Analyze the data comprehensively.
3. Provide actionable investment recommendations.

---
### Workflow Overview
1. Define custom tools to retrieve financial data, income statements, and stock prices.
2. Create agents for data gathering, analysis, and recommendation tasks.
3. Define tasks aligned with these agents' roles and objectives.
4. Configure a crew to execute these tasks sequentially, leveraging collaboration between agents.
5. Run the workflow and observe the results.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [ ]:
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from my_tools import get_current_stock_price, get_company_info, get_income_statements
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from a '.env' file
load_dotenv()

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Step 2: Define the Agents

In [ ]:
from crewai import Agent

# Agent for gathering company news and information
news_info_explorer = Agent(
    role='News and Info Researcher',
    goal='Gather and provide the latest news and information about a company from the internet',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company. '
        'Consider you are on: ' + Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=5,
)

# Agent for gathering financial data
data_explorer = Agent(
    role='Data Researcher',
    goal='Gather and provide financial data and company information about a stock',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company or stock. '
        'When using tools, add the suffix ".NS" to the stock symbol. '
        'Consider you are on: ' + Today
    ),
    tools=[get_company_info, get_income_statements],
    cache=True,
    max_iter=5,
)

# Agent for analyzing data
analyst = Agent(
    role='Data Analyst',
    goal='Consolidate financial data, stock information, and provide a summary',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are an expert in analyzing financial data, stock/company-related current information, and '
        'making a comprehensive analysis. Use Indian units for numbers (lakh, crore). '
        'Consider you are on: ' + Today
    ),
)

# Agent for financial recommendations
fin_expert = Agent(
    role='Financial Expert',
    goal='Considering financial analysis of a stock, make investment recommendations',
    llm='gpt-4o-mini',
    verbose=True,
    tools=[get_current_stock_price],
    max_iter=5,
    backstory=(
        'You are a financial advisor who can provide investment recommendations. '
        'Consider the financial analysis, current information about the company, current stock price, '
        'and make recommendations about whether to buy a stock. '
        'When using tools, add the suffix ".NS" to the stock symbol. '
        'Consider you are on: ' + Today
    ),
)

### Step 3: Define the Tasks

In [ ]:
from crewai import Task
from my_agents import news_info_explorer, data_explorer, analyst, fin_expert

# Task to gather financial data of a stock
get_company_financials = Task(
    description="Get financial data like income statements and other fundamental ratios for stock: {stock}",
    expected_output="Detailed information from income statement, key ratios for {stock}. "
                    "Indicate also about current financial status and trend over the period.",
    agent=data_explorer,
)

# Task to gather company news
get_company_news = Task(
    description="Get latest news and business information about company: {stock}",
    expected_output="Latest news and business information about the company. Provide a summary also.",
    agent=news_info_explorer,
)

# Task to analyze financial data and news
analyse = Task(
    description="Make thorough analysis based on given financial data and latest news of a stock",
    expected_output="Comprehensive analysis of a stock outlining financial health, stock valuation, risks, and news. "
                    "Mention currency information and number units in Indian context (lakh/crore).",
    agent=analyst,
    context=[get_company_financials, get_company_news],
    output_file='Analysis.md',
)

# Task to provide financial advice
advise = Task(
    description="Make a recommendation about investing in a stock, based on analysis provided and current stock price. "
                "State the reasons.",
    expected_output="Recommendation (Buy / No Buy) of a stock, with reasons clearly mentioned.",
    agent=fin_expert,
    context=[analyse],
    output_file='Recommendation.md',
)

### Step 4: Set Up the Crew

In [ ]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[data_explorer, news_info_explorer, analyst, fin_expert],
    tasks=[get_company_financials, get_company_news, analyse, advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 5: Run the Crew and Observe Results

In [ ]:
# Run the crew with a specific stock
result = crew.kickoff(inputs={'stock': 'Hdfc Bank'})

# Print the final result
print("Final Result:", result)


---
### Conclusion
This workflow demonstrates a detailed investment advisory process using multiple agents and tasks. The system showcases the integration of custom tools and collaborative agents in CrewAI to provide actionable financial insights.
